# 🎭 Emotion Word Expansion: WordNet vs NRC
This notebook demonstrates how to expand basic emotion terms using:

- **WordNet** (via NLTK) with low-drift controls
- **NRC Emotion Lexicon** (via NRCLex)
- Side-by-side comparison

You can use this for emotion modeling, lexicon building, or affective NLP.

In [1]:
# 📦 Install required packages
!pip install nltk nrclex --quiet


  DEPRECATION: Building 'nrclex' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'nrclex'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [9]:
# 📚 NLTK data setup
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('nrc')

[nltk_data] Downloading package wordnet to /Users/piek/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/piek/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Error loading nrc: Package 'nrc' not found in index


False

## 🔁 WordNet Expansion with Low Concept Drift

In [3]:
from nltk.corpus import wordnet as wn

def expand_emotion_wordnet(word):
    related = set()
    for pos in [wn.NOUN, wn.ADJ]:
        synsets = wn.synsets(word, pos=pos)
        if not synsets:
            continue

        syn = synsets[0]  # Most common meaning

        for lemma in syn.lemmas():
            related.add(lemma.name())
            for ant in lemma.antonyms():
                related.add(ant.name())
            for deriv in lemma.derivationally_related_forms():
                related.add(deriv.name())

        if pos == wn.ADJ:
            for similar in syn.similar_tos():
                for lemma in similar.lemmas():
                    related.add(lemma.name())

    return sorted(related)

## 🔍 NRC Emotion Lexicon (via NRCLex)

In [10]:
http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm

In [15]:
from collections import defaultdict

def load_nrc_lexicon(filepath):
    emotion_words = defaultdict(set)
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            word, emotion, score = line.strip().split('\t')
            if score == '1':
                emotion_words[emotion].add(word)
    return {emo: sorted(words) for emo, words in emotion_words.items()}

filepath = "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
nrc_emotion_lexicon = load_nrc_lexicon(filepath)


## ⚖️ Compare Emotion Expansions

In [16]:
basic_emotions = ['anger', 'joy', 'fear', 'sadness', 'surprise', 'disgust']

for emotion in basic_emotions:
    print(f"\n--- {emotion.upper()} ---")

    wn_words = expand_emotion_wordnet(emotion)
    print(f"WordNet ({len(wn_words)}): {', '.join(wn_words[:15])}...")

    nrc_words = nrc_emotion_lexicon[emotion]
    print(f"NRC     ({len(nrc_words)}): {', '.join(nrc_words[:15])}...")


--- ANGER ---
WordNet (5): anger, angry, choler, choleric, ire...
NRC     (1245): abandoned, abandonment, abhor, abhorrent, abolish, abomination, abuse, accursed, accusation, accused, accuser, accusing, actionable, adder, adversary...

--- JOY ---
WordNet (6): joy, joyful, joyfulness, joyous, joyousness, sorrow...
NRC     (687): absolution, abundance, abundant, accolade, accompaniment, accomplish, accomplished, achieve, achievement, acrobat, admirable, admiration, adorable, adoration, adore...

--- FEAR ---
WordNet (5): fear, fearful, fearfulness, fearlessness, fright...
NRC     (1474): abandon, abandoned, abandonment, abduction, abhor, abhorrent, abominable, abomination, abortion, absence, abuse, abyss, accident, accidental, accursed...

--- SADNESS ---
WordNet (5): happiness, sad, sadness, unhappiness, unhappy...
NRC     (1187): abandon, abandoned, abandonment, abduction, abortion, abortive, abscess, absence, absent, absentee, abuse, abysmal, abyss, accident, accursed...

--- SURPRI

## 📤 Export Results to CSV

In [19]:
import pandas as pd

all_data = []
for emotion in basic_emotions:
    wn_words = expand_emotion_wordnet(emotion)
    nrc_words = nrc_emotion_lexicon[emotion]

    all_data.append({
        'emotion': emotion,
        'wordnet_words': ", ".join(wn_words),
        'nrc_words': ", ".join(nrc_words)
    })

df = pd.DataFrame(all_data)
df.to_csv("emotion_expansion_comparison.csv", index=False)
df.head()

,emotion,wordnet_words,nrc_words
0,anger,"anger, angry, choler, choleric, ire","abandoned, abandonment, abhor, abhorrent, abol..."
1,joy,"joy, joyful, joyfulness, joyous, joyousness, s...","absolution, abundance, abundant, accolade, acc..."
2,fear,"fear, fearful, fearfulness, fearlessness, fright","abandon, abandoned, abandonment, abduction, ab..."
3,sadness,"happiness, sad, sadness, unhappiness, unhappy","abandon, abandoned, abandonment, abduction, ab..."
4,surprise,surprise,"abandonment, abduction, abrupt, accident, acci..."
